In [1]:
%load_ext autoreload
%autoreload 2
    
%load_ext tensorboard
import sys, os
sys.path.append('../paraphrase/')
from paraphraser_args import ModelArguments, DataTrainingArguments, TrainingArguments
from paraphraser_dataloader import load_dataset
from parpahraser_trainer import ParaphraserTrainer
from transformers import AutoTokenizer, AutoModelWithLMHead, HfArgumentParser

In [2]:
data_dir = "../data/paranmt_filtered"
model_name = "t5-base"
model_nick = "t5"
output_dir = "../models/"
epochs = "3"
train_batch_size = "2"
eval_batch_size = "2"

parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = parser.parse_args_into_dataclasses([
    "--model_name_or_path",
    model_name,
    "--model_nick",
    model_nick,
    "--data_dir",
    data_dir,
    "--output_dir",
    os.path.join(output_dir, model_nick),
    "--cache_dir",
    os.path.join(output_dir,"cache"),
    "--overwrite_cache",
    "--per_device_train_batch_size",
    train_batch_size,
    "--per_device_eval_batch_size",
    eval_batch_size,
    "--max_seq_len",
    "64",
    "--gradient_accumulation_steps",
    "1",
    "--num_train_epochs",
    epochs,
    "--logging_steps",
    "100",
    "--save_steps",
    "100"
])


PyTorch: setting up devices
/home/nuwandavek/anaconda3/envs/marvin/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelWithLMHead.from_pretrained("t5-base")

loading configuration file https://huggingface.co/t5-base/resolve/main/config.json from cache at /home/nuwandavek/.cache/huggingface/transformers/91e9fe874e06c44883b535d6c950b8b89d6eaa3298d8e7fb3b2c78039e9f8b7b.66b9637a52aa11e9285cdd6e668cc0df14b3bcf0b6674cf3ba5353c542649637
Model config T5Config {
  "architectures": [
    "T5WithLMHeadModel"
  ],
  "d_ff": 3072,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 

In [4]:
train_dataset = load_dataset(data_args.data_dir, tokenizer, mode="train", n_proc=1024)
dev_dataset = load_dataset(data_args.data_dir, tokenizer, mode="dev", n_proc=1024)

100%|██████████| 2/2 [00:00<00:00,  9.95it/s]


In [5]:
# Open TensorBoard
%tensorboard --logdir runs

Reusing TensorBoard on port 6006 (pid 123206), started 0:12:10 ago. (Use '!kill 123206' to kill it.)

In [ ]:
trainer = ParaphraserTrainer([training_args,model_args, data_args], model, tokenizer, train_dataset, dev_dataset)
trainer.train()

***** Running training *****
Num examples = <torch.utils.data.dataset.TensorDataset object at 0x7f5b6a7f04f0>
Num Epochs = 3.0
Total train batch size = 2
Gradient Accumulation steps = 1
Total optimization steps = 109593.0
Logging steps = 100
Save steps = 100
